<a href="https://colab.research.google.com/github/CharlyPierce/Basico/blob/main/feture_columns_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
#title Run to load and clean the dataset
%reset -f
from __future__ import print_function

import math
import numpy as np
import numpy.linalg as nla
import pandas as pd
import re
import six
from os.path import join
from matplotlib import pyplot as plt
# Set the output display to have one digit for decimal places and limit it to
# printing 15 rows.rs=tf.feature_column.indicator_column(sf_column)
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_rows = 15

choc_data = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/flavors_of_cacao.csv", sep=",", encoding='latin-1')
# We can rename the columns.
choc_data.columns = ['maker', 'specific_origin', 'reference_number', 'review_date', 'cocoa_percent', 'maker_location', 
                     'rating', 'bean_type', 'broad_origin']
# choc_data.dtypes
# Replace empty/null values with "Blend"
choc_data['bean_type'] = choc_data['bean_type'].fillna('Blend')
#title Cast bean_type to string to remove leading 'u'
choc_data['bean_type'] = choc_data['bean_type'].astype(str)
choc_data['cocoa_percent'] = choc_data['cocoa_percent'].str.strip('%')
choc_data['cocoa_percent'] = pd.to_numeric(choc_data['cocoa_percent'])
#title Correct spelling mistakes, and replace city with country name
choc_data['maker_location'] = choc_data['maker_location']\
.str.replace('Amsterdam', 'Holland')\
.str.replace('U.K.', 'England')\
.str.replace('Niacragua', 'Nicaragua')\
.str.replace('Domincan Republic', 'Dominican Republic')
# Adding this so that Holland and Netherlands map to the same country.
choc_data['maker_location'] = choc_data['maker_location']\
.str.replace('Holland', 'Netherlands')

def cleanup_spelling_abbrev(text):
    replacements = [
        ['-', ', '], ['/ ', ', '], ['/', ', '], ['\(', ', '], [' and', ', '], [' &', ', '], ['\)', ''],
        ['Dom Rep|DR|Domin Rep|Dominican Rep,|Domincan Republic', 'Dominican Republic'],
        ['Mad,|Mad$', 'Madagascar, '],
        ['PNG', 'Papua New Guinea, '],
        ['Guat,|Guat$', 'Guatemala, '],
        ['Ven,|Ven$|Venez,|Venez$', 'Venezuela, '],
        ['Ecu,|Ecu$|Ecuad,|Ecuad$', 'Ecuador, '],
        ['Nic,|Nic$', 'Nicaragua, '],
        ['Cost Rica', 'Costa Rica'],
        ['Mex,|Mex$', 'Mexico, '],
        ['Jam,|Jam$', 'Jamaica, '],
        ['Haw,|Haw$', 'Hawaii, '],
        ['Gre,|Gre$', 'Grenada, '],
        ['Tri,|Tri$', 'Trinidad, '],
        ['C Am', 'Central America'],
        ['S America', 'South America'],
        [', $', ''], [',  ', ', '], [', ,', ', '], ['\xa0', ' '],[',\s+', ','],
        [' Bali', ',Bali']
    ]
    for i, j in replacements:
        text = re.sub(i, j, text)
    return text

choc_data['specific_origin'] = choc_data['specific_origin'].str.replace('.', '').apply(cleanup_spelling_abbrev)

#title Cast specific_origin to string
choc_data['specific_origin'] = choc_data['specific_origin'].astype(str)

#title Replace null-valued fields with the same value as for specific_origin
choc_data['broad_origin'] = choc_data['broad_origin'].fillna(choc_data['specific_origin'])

# title Clean up spelling mistakes and deal with abbreviations
choc_data['broad_origin'] = choc_data['broad_origin'].str.replace('.', '').apply(cleanup_spelling_abbrev)

# Change 'Trinitario, Criollo' to "Criollo, Trinitario"
# Check with choc_data['bean_type'].unique()
choc_data.loc[choc_data['bean_type'].isin(['Trinitario, Criollo']),'bean_type'] = "Criollo, Trinitario"
# Confirm with choc_data[choc_data['bean_type'].isin(['Trinitario, Criollo'])]

# Fix chocolate maker names
choc_data.loc[choc_data['maker']=='Shattel','maker'] = 'Shattell'
choc_data['maker'] = choc_data['maker'].str.replace(u'Na\xef\xbf\xbdve','Naive')

choc_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


,maker,specific_origin,reference_number,review_date,cocoa_percent,maker_location,rating,bean_type,broad_origin
0,A. Morin,Agua Grande,1876,2016,63.00,France,3.75,Blend,Sao Tome
1,A. Morin,Kpime,1676,2015,70.00,France,2.75,Blend,Togo
2,A. Morin,Atsane,1676,2015,70.00,France,3.00,Blend,Togo
3,A. Morin,Akata,1680,2015,70.00,France,3.50,Blend,Togo
4,A. Morin,Quilla,1704,2015,70.00,France,3.50,Blend,Peru


In [ ]:
sparse_input_feature_embedding_dims={'bean_type': 5, 'broad_origin': 7, 'maker': 9, 'maker_location': 6}
sparse_feature_vocabs = {'maker': sorted(list(set(choc_data['maker'].values))) }
sf_column = tf.feature_column.categorical_column_with_vocabulary_list(key='maker',vocabulary_list=sparse_feature_vocabs['maker'],num_oov_buckets=0)
f_columns=tf.feature_column.embedding_column(categorical_column=sf_column,dimension=sparse_input_feature_embedding_dims['maker'],
                                   combiner='mean',)
f_columns

In [ ]:
rs=tf.feature_column.indicator_column(sf_column)
rs1=tf.keras.layers.DenseFeatures(f_columns,)

In [ ]:
a=rs1(dict(choc_data))
a

<tf.Tensor: shape=(1795, 9), dtype=float32, numpy=
array([[-0.23497796,  0.08820882, -0.05156199, ..., -0.05253533,
        -0.13444021,  0.30908722],
       [-0.23497796,  0.08820882, -0.05156199, ..., -0.05253533,
        -0.13444021,  0.30908722],
       [-0.23497796,  0.08820882, -0.05156199, ..., -0.05253533,
        -0.13444021,  0.30908722],
       ...,
       [-0.06998488,  0.25316417,  0.50854385, ...,  0.27477047,
         0.51148176,  0.1509257 ],
       [-0.06998488,  0.25316417,  0.50854385, ...,  0.27477047,
         0.51148176,  0.1509257 ],
       [-0.06998488,  0.25316417,  0.50854385, ...,  0.27477047,
         0.51148176,  0.1509257 ]], dtype=float32)>

In [ ]:
choc_data['key1']=choc_data['maker'].index.values

In [ ]:
# s2={name: value for name,value in choc_data.items()}
# rs=tf.feature_column.indicator_column(sf_column)
type(tf.feature_column.categorical_column_with_vocabulary_list(key='maker',vocabulary_list=sparse_feature_vocabs['maker'],num_oov_buckets=0))

tensorflow.python.feature_column.feature_column_v2.VocabularyListCategoricalColumn